# Exercise 3: Decision Tree

---
**Written by Hendi Lie (h2.lie@qut.edu.au) and Richi Nayak (r.nayak@qut.edu.au). All rights reserved.**

Welcome to the third practical exercise for IFN645. Each exercise sheet contains a number of theoretical and programming exercises, designed to strengthen both conceptual and practical understanding of data mining processes in this unit.

To answer conceptual questions, write the answer to each question on a paper/note with your reasoning. For programming exercises, open your iPython console/Jupyter notebook and use Python commands/libraries introduced in each practical to answer the questions. In many cases, you will need to write code to support your conceptual answers.

## 0. Prequisite

Perform the following steps before trying the exercises:
1. Import pandas as "pd" and load the house price dataset into "df".
2. Print dataset information to refresh your memory.
3. Run `preprocess_data` function on the dataframe to perform preprocessing steps discussed last week.

In [8]:
import pandas as pd

df = pd.read_csv('datasets/melbourne_house_price.csv', index_col=0)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24197 entries, 0 to 24196
Data columns (total 22 columns):
Suburb                24197 non-null object
Address               24197 non-null object
Rooms                 24197 non-null int64
Type                  24197 non-null object
Price                 24197 non-null float64
Method                24197 non-null object
SellerG               24197 non-null object
Date                  24197 non-null object
Distance              24196 non-null float64
Postcode              24196 non-null float64
Bedroom2              18673 non-null float64
Bathroom              18669 non-null float64
Car                   18394 non-null float64
Landsize              15946 non-null float64
BuildingArea          9609 non-null float64
YearBuilt             10961 non-null float64
CouncilArea           24194 non-null object
Lattitude             18843 non-null float64
Longtitude            18843 non-null float64
Regionname            24194 non-null object
Pr

In [18]:
def preprocess_data(df):
    # Q1.4 and Q6.2
    df = df.drop(['Address', 'Landsize', 'BuildingArea', 'YearBuilt', 'Price', 'Bedroom2', 'SellerG'], axis=1)
    
    # Q1.1
    cols_miss_drop =['Postcode', 'CouncilArea', 'Regionname', 'Propertycount']
    mask = pd.isnull(df['Distance'])

    for col in cols_miss_drop:
        mask = mask | pd.isnull(df[col])

    df = df[~mask]
    
    # Q1.2
    df['Bathroom'].fillna(df['Bathroom'].mean(), inplace=True)
    df['Car'].fillna(df['Car'].mean(), inplace=True)
    
    df['Latitude_nan'] = pd.isnull(df['Lattitude'])
    df['Longtitude_nan'] = pd.isnull(df['Longtitude'])
    df['Lattitude'].fillna(0, inplace=True)
    df['Longtitude'].fillna(0, inplace=True)
    
    # Q6.1. Change date into weeks and months
    df['Sales_week'] = pd.to_datetime(df['Date']).dt.week
    df['Sales_month'] = pd.to_datetime(df['Date']).dt.month
    df = df.drop(['Date'], axis=1)  # drop the date, not required anymore
    
    # Q4
    df = pd.get_dummies(df)
    
    return df

df2 = preprocess_data(df)

In [19]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24194 entries, 0 to 24196
Columns: 402 entries, Rooms to Regionname_Western Victoria
dtypes: bool(2), float64(7), int64(4), uint8(389)
memory usage: 11.2 MB


In [20]:

"""pd.set_option('display.height', 60)
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 80)"""


"pd.set_option('display.height', 60)\npd.set_option('display.max_rows', 60)\npd.set_option('display.max_columns', 20)\npd.set_option('display.width', 80)"

## 1. Data Partitioning

Perform following operations and answer the following questions:
1. Describe training, validation and test dataset. What is the purpose for each of these split?
    * Training set is used for training the model. 
    * The validation set is used for testing the model against "unseen" data
    * test set is used for final accuracy testing
2. What is k-fold cross validation? What is the advantage and disadvantage of k-fold CV compared to normal training/test/validation method?
    * k-fold cross validation is randomly splitting the data set into k equal sized partition, where the one part will be used for validation while the rest is used for training. advantage is 
3. What does it mean by *stratification*?
    * Stratification ensures same ratio of positive and negative targets in train and test data sets
4. What does random state do?
    * Random number generator

5. Set random state to 0. Split the dataframe into X and Y, then split respective data into training and test set of 70/30 proportion.

In [21]:
from sklearn.model_selection import train_test_split

y = df2['Price_above_median']
x = df2.drop(['Price_above_median'], axis=1)

rs = 0
x_mat = x.as_matrix()
x_train, x_test, y_train, y_test = train_test_split(x_mat, y, test_size=0.3, stratify=y, random_state=rs)



## 2. Decision Tree

Perform the following operations and answer the question.
1. Import and build a decision tree classifier. Set the random state to 0 to ensure your result is similar with the answers. Fit it against the training data.
2. What is the performance of the model against training data? How about against the test data? Do you see any indication of overfitting here?
3. What are the top 5 most important features in this model?
4. Find the best hyperparameters using GridSearchCV. What is the optimal parameter set? Use the following parameters as initial guidance **criterion** of `gini` or `entropy`, **max depth** of 2-7 and **min_samples_leaf** from 20-60, increment of 10.
    
5. Visualise the structure of your decision tree. Can you identify characteristics of expensive houses?

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# simple decision tree training
model = DecisionTreeClassifier(random_state=rs)
#model = DecisionTreeClassifier(criterion='gini', random_state=rs, max_depth=2, min_samples_leaf=20)
model.fit(x_train, y_train)



DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [23]:
print("Train accuracy:", model.score(x_train, y_train))
print("Test accuracy:", model.score(x_test, y_test))

Train accuracy: 0.9981694715087098
Test accuracy: 0.8425402948064472


In [24]:
import numpy as np

# grab feature importances from the model and feature name from the original X
importances = model.feature_importances_
feature_names = x.columns

# sort them out in descending order
indices = np.argsort(importances)
indices = np.flip(indices, axis=0)

# limit to 20 features, you can leave this out to print out everything
indices = indices[:5]

for i in indices:
    print(feature_names[i], ':', importances[i])

Type_u : 0.1663717197004314
Distance : 0.14105464031286377
Regionname_Southern Metropolitan : 0.13320735698073669
Longtitude : 0.0735364961075368
Regionname_Eastern Metropolitan : 0.07287281292272205


In [27]:
from sklearn.model_selection import GridSearchCV

In [28]:
# grid search CV
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(2, 7),
          'min_samples_leaf': range(20, 60, 10)}

cv = GridSearchCV(param_grid=params, estimator=DecisionTreeClassifier(random_state=rs), cv=10)
cv.fit(x_train, y_train)

print("Train accuracy:", cv.score(x_train, y_train))
print("Test accuracy:", cv.score(x_test, y_test))

# test the best model
y_pred = cv.predict(x_test)
print(classification_report(y_test, y_pred))

# print parameters of the best model
print(cv.best_params_)

Train accuracy: 0.8577502214348981
Test accuracy: 0.8501170960187353
             precision    recall  f1-score   support

          0       0.84      0.87      0.85      3634
          1       0.86      0.83      0.85      3625

avg / total       0.85      0.85      0.85      7259

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 40}


In [29]:
model = DecisionTreeClassifier(criterion='gini', random_state=rs, max_depth=6, min_samples_leaf=40)
model.fit(x_train, y_train)

print("Train accuracy:", model.score(x_train, y_train))
print("Test accuracy:", model.score(x_test, y_test))

Train accuracy: 0.8577502214348981
Test accuracy: 0.8501170960187353


In [30]:
import pydot
from io import StringIO
from sklearn.tree import export_graphviz

# visualize
dotfile = StringIO()
export_graphviz(model, out_file=dotfile, feature_names=x.columns)
graph = pydot.graph_from_dot_data(dotfile.getvalue())
graph[0].write_png("exercise3.png") # saved in the following file - will return True if successful

# Answer

When you are finished with all exercise questions, the sample answers are available in the following Github repository. Remember, please try the exercises first before viewing the answers.

https://github.com/liehendi11/IFN645_answers